<a href="https://colab.research.google.com/github/antounes/deep-learning-fastai-pytorch/blob/main/02_from_model_to_production_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Bicycle Spare Parts with Deep Learning - A Trial

This is the first post of my research work on convolutional neural networks, a family of neural networks mostly used (but not only) for solving computer vision problems. This personal research work should help me getting a sound understanding of how convolutional neural networks work under the hood. Notebooks help mixing computer science practice and deep learning theory.

This notebook is implemented with fast.ai and PyTorch, and is greatly inspired from Howard's and Gugger's book *Deep Learning for Coders with fast.ai and PyTorch*.

## Introduction

The goal here is to build a classifier for distinguishing between bicycle spare parts. When looking for parts, one of the most tedious tasks is to differentiate between different versions of a given part called a cassette. The cassette is a block of sprockets located on the bicycle rear wheel and ensuring the transmission of the power applied on the pedals to the rear wheel via the chain, hence making the bike moving forward. Most today's bicycle drivetrains use either 10-, 11- or 12-sprocket cassettes. Distinguishing between these at first glance is not easy, as can be seen below. In fact what we just see is just a mass of cogs.


A 10-speed cassette  | An 11-speed cassette | A 12-speed cassette
-------------------  |----------------------|--------------------
<img src="https://drive.google.com/uc?id=1f8_sMjLukd4po_sZ9ZjIgjUswH9mGd4L" width="200"> | <img src="https://drive.google.com/uc?id=1qrQc14VwHFYy587n6bWH55eIkWCCaMBL" width="200"> | <img src="https://drive.google.com/uc?id=1c2DCHRV2RamZGjvTJp9JwVqYUgOx7Avp" width="155">


Distinguishing between types of cassettes is crucial though, because not all cassettes can be used with all chains and all shifters on a bicycle. 10-speed cassettes go with 10-speed drivetrains, etc.

On could argue that it's just about counting the sprockets, or reading the numbers written on the lockring (the smallest and darkest ring of the cassette), but it's not always as easy, especially with greasy or muddy parts, or even with old ones where nothing can be read anymore.

Let's assess if such objects are understandable and can be represented in such a manner that a neural network can distinguish between their types.

## Gathering Data

First we need to install some utilities provided by fast.ai authors, namely the fastbook library. This library comes with classes and modules that will help us setting up deep learing models.

In [1]:
!pip install -Uqq fastbook

In [2]:
import fastbook
fastbook.setup_book()
from fastbook import *

fast.ai authors have built a tool that allows to download images from search engines, thus ensuring we can build our image dataset. Here we'll be using the most up-to-date version of the class, scrapping images from DuckDuckGo search results.
For the sake of time-saving and reproducibility, we'll limit the dataset to 100 images of each type of cassette we're trying to identify.

Images are scrapped from the Internet and stored in a folder created for each type of cassette. Some additional steps consist of checking if all images are in the create folders and if there are no corrupted files, since they've been downloaded from the Internet. Corrupted files are eventually discarded.

In [3]:
# Download URLs for all of our search terms
cassettes = ('10 speed', '11 speed', '12 speed')
path = Path('cassettes')

if not path.exists():
  path.mkdir()
for c in cassettes:
  dest = (path/c)
  dest.mkdir(exist_ok=True)
  results = search_images_ddg('{} bicycle cassette'.format(c), max_images=100)
  download_images(urls=results, dest=dest)

In [4]:
# Check images are located in the destination folder
fns = get_image_files(path)
print('{} downloaded files:'.format(len(fns)), fns)

# Check if some files are corrupt
failed = verify_images(fns)
print('{} corrupted file(s):'.format(len(failed)), failed)

# Remove failed images
failed.map(Path.unlink);

384 downloaded files: [Path('cassettes/12 speed/00000067.jpg'), Path('cassettes/12 speed/00000088.jpg'), Path('cassettes/12 speed/00000073.jpg'), Path('cassettes/12 speed/00000037.jpg'), Path('cassettes/12 speed/00000008.jpeg'), Path('cassettes/12 speed/00000038.jpg'), Path('cassettes/12 speed/00000077.jpeg'), Path('cassettes/12 speed/00000012.jpg'), Path('cassettes/12 speed/00000057.jpg'), Path('cassettes/12 speed/00000023.jpg'), Path('cassettes/12 speed/00000085.jpg'), Path('cassettes/12 speed/00000014.jpg'), Path('cassettes/12 speed/00000064.jpg'), Path('cassettes/12 speed/00000060.jpeg'), Path('cassettes/12 speed/00000011.jpg'), Path('cassettes/12 speed/00000075.jpg'), Path('cassettes/12 speed/00000080.jpg'), Path('cassettes/12 speed/00000026.jpg'), Path('cassettes/12 speed/00000066.jpeg'), Path('cassettes/12 speed/00000002.jpg'), Path('cassettes/12 speed/00000003.jpg'), Path('cassettes/12 speed/00000070.jpg'), Path('cassettes/12 speed/00000021.jpg'), Path('cassettes/12 speed/00000

## From Data to DataLoaders